In [33]:
#CS5228 project

# data manipulation
import pandas as pd
import numpy as np

# visualiation
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline

# model training
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# classifiers
from sklearn.naive_bayes import GaussianNB # naive bayes
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.svm import SVC # SVM
from sklearn.linear_model import LogisticRegression # logistic regression
from sklearn.tree import DecisionTreeClassifier # decision Tree
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import RFE # for feature selection of LR
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler

# ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [27]:
#load items into dataframe
financial_data_raw = pd.read_csv('financial_data.csv', na_values=['?'])
available_training_truth = pd.read_csv('revealed_businesses.csv')
testing_data = pd.read_csv('testing_data.csv')

In [20]:
len(financial_data_raw)

9003

In [16]:
cID = testing_data['Var1'].tolist()

In [14]:
#print(len(financial_data_raw))
#print(financial_data_raw.head())
#print(financial_data_raw.dtypes)

print(len(available_training_truth))
print(available_training_truth.head())
print(available_training_truth.dtypes)
training_results_series = available_training_truth['Var66']
print(len(training_results_series)) #total we only have 4879 given truths
print(len([x for x in training_results_series if x == 0])) # 4709 of them are 0
print(len([x for x in training_results_series if x == 1])) # 170 of them are 1

4879
    Var1  Var66
0  18399      0
1  19821      0
2  17769      0
3  19309      0
4  20728      0
Var1     int64
Var66    int64
dtype: object
4879
4709
170


In [28]:
training_data_with_label = financial_data_raw.merge(available_training_truth, on=('Var1')) 
# print('length of training data iwth labels: ' + str(len(training_data_with_label)))
# print(training_data_with_label.head())
# print(type(training_data_with_label))
# #print(len(training_data_with_label))
print(training_data_with_label.info())

coys_with_labels = training_data_with_label['Var1'].tolist()

print('financial data raw')
print(len(financial_data_raw))

coys_with_no_label = []

for index, row in financial_data_raw.iterrows():
    if row['Var1'] in coys_with_labels:
        continue
    coys_with_no_label += [row['Var1']]
    
print(type(coys_with_no_label))
print(len(coys_with_no_label))
#print(coys_with_no_label)

training_data_with_no_label = financial_data_raw.loc[financial_data_raw['Var1'].isin(coys_with_no_label)]

print(type(training_data_with_no_label))
print(len(training_data_with_no_label))
print(training_data_with_no_label.head())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4879 entries, 0 to 4878
Data columns (total 66 columns):
Var1     4879 non-null int64
Var2     4879 non-null float64
Var3     4879 non-null float64
Var4     4879 non-null float64
Var5     4870 non-null float64
Var6     4870 non-null float64
Var7     4879 non-null float64
Var8     4879 non-null float64
Var9     4872 non-null float64
Var10    4879 non-null float64
Var11    4879 non-null float64
Var12    4879 non-null float64
Var13    4870 non-null float64
Var14    4863 non-null float64
Var15    4879 non-null float64
Var16    4877 non-null float64
Var17    4872 non-null float64
Var18    4872 non-null float64
Var19    4879 non-null float64
Var20    4863 non-null float64
Var21    4863 non-null float64
Var22    4540 non-null float64
Var23    4879 non-null float64
Var24    4863 non-null float64
Var25    4879 non-null float64
Var26    4879 non-null float64
Var27    4872 non-null float64
Var28    4563 non-null float64
Var29    4781 non-null floa

In [30]:
#clean up the training data with no label

training_data_with_no_label.isnull().sum()

Var1        0
Var2        0
Var3        0
Var4        0
Var5        7
Var6        9
Var7        0
Var8        0
Var9        5
Var10       0
Var11       0
Var12       0
Var13       7
Var14      17
Var15       0
Var16       4
Var17       5
Var18       5
Var19       0
Var20      17
Var21      17
Var22     319
Var23       0
Var24      17
Var25       0
Var26       0
Var27       5
Var28     251
Var29     103
Var30       0
         ... 
Var36       0
Var37       0
Var38    1980
Var39       0
Var40      17
Var41       7
Var42      94
Var43      17
Var44      17
Var45      17
Var46     217
Var47       7
Var48      43
Var49       0
Var50      17
Var51       5
Var52       0
Var53      43
Var54     103
Var55     103
Var56       0
Var57      17
Var58       0
Var59      10
Var60       0
Var61     218
Var62       7
Var63      17
Var64       7
Var65     103
Length: 65, dtype: int64

In [31]:
#clean data
training_data_with_no_label = training_data_with_no_label.replace(0, np.nan) # assume 0 values also means missing
training_data_with_no_label = training_data_with_no_label.fillna(training_data_with_no_label.median())

In [32]:
#raw_input()
#do KMeans
from sklearn.cluster import *

clustering_training = KMeans(n_clusters=2, random_state=0).fit(training_data_with_no_label)
y_predicted = clustering_training.predict(training_data_with_no_label)
print(len(y_predicted))
print(type(y_predicted))
print(y_predicted)
print(type(clustering_training))

4124
<class 'numpy.ndarray'>
[0 0 0 ... 0 0 0]
<class 'sklearn.cluster.k_means_.KMeans'>


In [42]:
aug_training_truth_0 = []

for a, b in zip(training_data_with_no_label['Var1'].tolist(), y_predicted):
    aug_training_truth_0 += [(a, b)]
print(type(aug_training_truth_0))
print(len(aug_training_truth_0))

<class 'list'>
4124


In [44]:
aug_training_truth_0[1]

(14171, 0)

In [45]:
aug_training_truth_1 = []

for a, b in zip(training_data_with_no_label['Var1'].tolist(), y_predicted):
    if b==1 or b=='1':
        aug_training_truth_1 += [(a, 0)]
    else:
        aug_training_truth_1 += [(a, 1)]
print(type(aug_training_truth_1))
print(len(aug_training_truth_1))

<class 'list'>
4124


In [46]:
aug_training_truth_1[1]

(14171, 1)

In [51]:
augmented_training_truth_1 = pd.DataFrame(aug_training_truth_1, columns=['Var1', 'Var66'])
augmented_training_truth_0 = pd.DataFrame(aug_training_truth_0, columns=['Var1', 'Var66'])

print(augmented_training_truth_1.loc[1])
print(augmented_training_truth_0.loc[1])

Var1     14171
Var66        1
Name: 1, dtype: int64
Var1     14171
Var66        0
Name: 1, dtype: int64


In [55]:
print(len(available_training_truth))

4879


In [53]:
#write out data to augmented_revealed_businesses
#df.append(df2, ignore_index = True) 
full_training_truth_1 = available_training_truth.append(augmented_training_truth_1, ignore_index=True)
full_training_truth_0 = available_training_truth.append(augmented_training_truth_0, ignore_index=True)

In [57]:
print(full_training_truth_1.loc[4879])
print(full_training_truth_0.loc[4979])

Var1     15092
Var66        1
Name: 4879, dtype: int64
Var1     13990
Var66        0
Name: 4979, dtype: int64


In [58]:
p = open('augmented_revealed_businesses_0.csv', 'w')
p.write('Var1, Var66\n')
for index, row in full_training_truth_0.iterrows():
    p.write(str(row['Var1']) + ',' + str(row['Var66']) + '\n')
p.close()

In [59]:
p = open('augmented_revealed_businesses_1.csv', 'w')
p.write('Var1, Var66\n')
for index, row in full_training_truth_1.iterrows():
    p.write(str(row['Var1']) + ',' + str(row['Var66']) + '\n')
p.close()

In [36]:
#do Agglomerative
#clustering_training = KMeans(n_clusters=2, random_state=0).fit(training_data_with_no_label)
agglo_clustering = AgglomerativeClustering()
y_agglo_predicted = agglo_clustering.fit_predict(training_data_with_no_label)
print(len(y_agglo_predicted))
print(type(y_agglo_predicted))
print(y_agglo_predicted)
#print(type(clustering_training))

4124
<class 'numpy.ndarray'>
[0 0 0 ... 0 0 0]


In [37]:
len([x for x in y_agglo_predicted if x ==1])

1

In [34]:
#load into dataframe
fdata = pd.read_csv('financial_data.csv', na_values=['?']) # ? means missing
revealed = pd.read_csv('augmented_revealed_businesses_0.csv')
tdata = pd.read_csv('testing_data.csv', na_values=['?'])

display(fdata.head())
display(revealed.head())

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var56,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65
0,18399,0.023954,0.150120,0.395670,3.63570,54.043,0.028822,0.031029,4.56831,1.01120,...,3871.001,0.011041,0.034914,0.98896,0.000000,9.5214,5.8248,34.713,10.5150,3.4752
1,15092,0.049699,0.065808,0.726800,12.94400,233.110,0.000000,0.063192,14.19601,0.89618,...,8751.901,0.059565,0.053189,0.93169,0.000000,5.0492,11.1520,24.784,14.7270,4.2204
2,19821,-0.356310,0.392880,0.158840,1.40430,-2.619,-0.085597,-0.356320,1.54531,0.92963,...,44.859,-0.172770,-0.586910,1.38330,0.000000,5.6112,15.7790,154.260,2.3662,2.0738
3,14171,0.001417,0.708110,-0.052312,0.88978,-31.198,0.269520,0.001407,0.41222,1.96540,...,-331.879,-0.000535,0.004820,0.99930,0.745480,17.1011,7.9482,88.147,4.1408,3.4021
4,12900,0.020041,0.346520,0.335930,2.76130,39.050,0.000000,0.020031,1.88591,1.29750,...,38170.001,0.212410,0.030652,0.80158,0.000862,9.7670,6.7570,53.651,6.8032,2.7412


,Var1,Var66
0,18399,0
1,19821,0
2,17769,0
3,19309,0
4,20728,0


In [35]:
#clean data
#fdata = fdata.replace(0, np.nan) # assume 0 values also means missing
fdata = fdata.fillna(fdata.median())
#tdata = tdata.replace(0, np.nan)
tdata = tdata.fillna(tdata.median())



In [36]:
#preprocess data
fdatan = fdata.copy(deep=True)
normald = StandardScaler()
fdatan_1 = normald.fit_transform(fdatan.ix[:,fdatan.columns!="Var1"]) # normalize all variables except Var1
fdatan = pd.DataFrame(np.column_stack((fdatan["Var1"].values,fdatan_1)),columns = fdatan.columns).set_index(fdatan.index)
tdatan = tdata.copy(deep=True)
normald = StandardScaler()
tdatan_1 = normald.fit_transform(tdatan.ix[:,tdatan.columns!="Var1"]) # normalize all variables except Var1
tdatan = pd.DataFrame(np.column_stack((tdatan["Var1"].values,tdatan_1)),columns = tdatan.columns).set_index(tdatan.index)



In [37]:
#merge dataframe
fdatan_merged = fdatan.merge(revealed, how="outer")
fdatan_train1 = fdatan.merge(revealed) # dataframe, whose bankruptcy status is known (0 = good standing, 1 = bankrupt)



In [18]:
fdatan_train1.head()


,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var57,Var58,Var59,Var60,Var61,Var62,Var63,Var64,Var65,Var66
0,18399.0,-0.048725,-0.085637,0.051995,-0.012700,0.012603,0.018969,-0.057168,-0.019384,-0.104092,...,0.010499,0.001866,-0.017767,-0.017397,-0.015475,-0.099020,-0.002138,0.042476,-0.075742,0
1,15092.0,-0.009047,-0.102170,0.117008,0.003775,0.014003,0.018698,-0.007847,-0.005803,-0.118195,...,0.011304,0.002780,-0.018530,-0.017397,-0.015590,-0.037611,-0.002496,0.179782,-0.074059,0
2,19821.0,-0.634787,-0.038035,0.005497,-0.016649,0.012160,-0.001816,-0.651161,-0.023648,-0.114093,...,0.007453,-0.029253,-0.012510,-0.017397,-0.015575,0.015727,0.002166,-0.223167,-0.078907,0
3,14171.0,-0.083459,0.023778,-0.035960,-0.017560,0.011936,0.062693,-0.102593,-0.025246,0.012908,...,0.010308,0.000360,-0.017629,-0.010754,-0.015281,-0.074543,-0.000214,-0.165317,-0.075907,0
4,12900.0,-0.054756,-0.047126,0.040266,-0.014247,0.012485,0.018698,-0.074033,-0.023168,-0.068987,...,0.013837,0.001652,-0.020264,-0.019690,-0.015469,-0.088274,-0.001456,-0.078525,-0.077400,0


In [20]:
fdatan_train1.dtypes[-4:]

Var63     float64
Var64     float64
Var65     float64
 Var66      int64
dtype: object

In [38]:
#classifier here
x1 = fdatan_train1.drop(columns=['Var1', ' Var66']) # Var 1 is company ID, Var 66 is the status
new_x1 = fdatan_train1[['Var18','Var19', 'Var46', 'Var59', 'Var51', 'Var52']]
y1 = fdatan_train1[' Var66']
t1 = tdata.drop(columns=['Var1'])
new_t1 = tdata[['Var18','Var19', 'Var46', 'Var59', 'Var51', 'Var52']]
cID = tdata['Var1'].tolist() # use original values, since nothing is done in cID of tdata
from sklearn.svm import SVC
#clf = SVC(gamma='auto')
#clf.fit(X, y) 
clas = AdaBoostClassifier()#SVC(gamma='auto')#GradientBoostingClassifier(learning_rate=0.1)#GaussianNB() #KNeighborsClassifier(n_neighbors=100) #SVC(gamma='auto') #AdaBoostClassifier() #  ## select classifier here
clas.fit(new_x1, y1)

# write results to predictions.csv
f = open('predictions_ada_aug_keepzero.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a,b in zip(cID, clas.predict(new_t1)):
    f.write(str(a))
    f.write(',')
    f.write(str(round(b)))
    f.write('\n')
f.close()

In [17]:
# write results to predictions.csv (baseline)
f = open('predictions_baseline_zero.csv', 'w')
f.write('Business_ID,Is_Bankrupted\n')
for a in cID:
    f.write(str(a))
    f.write(',')
    f.write(str(0))
    f.write('\n')
f.close()

In [15]:
#making the augmented revealed businesses
from sklearn.cluster import *
fdata_unprocessed = pd.read_csv('financial_data.csv', na_values=['?']) # ? means missing
revealed_unprocessed = pd.read_csv('revealed_businesses.csv')

clustering_training = KMeans(n_clusters=2, random_state=0).fit(x1)
y_predicted = clustering_training.predict(x1)
print(type(clustering_training))



<class 'sklearn.cluster.k_means_.KMeans'>


In [21]:
len(fdata_unprocessed)

9003

In [16]:
print(y_predicted)

[0 0 0 ... 0 0 0]


In [20]:
len(x1)

4879

In [75]:
new_x1.head()

,Var18,Var19,Var46,Var59,Var51,Var52
0,-0.018310,-0.050119,-0.010033,-0.017767,-0.010526,-0.061445
1,-0.024077,-0.499019,-0.011306,-0.012510,-0.014483,-0.013760
2,-0.025706,-0.081339,-0.010080,-0.017664,-0.016354,0.051094
3,-0.011013,-0.052824,-0.009965,-0.017711,-0.007064,-0.074384
4,-0.024283,-0.027177,-0.010015,-0.017500,-0.014818,-0.018443


In [8]:
len(revealed[revealed['Var66'].isnull()])

0

In [30]:
revealed[100:120]

,Var1,Var66
100,21282,0
101,21568,1
102,21085,0
103,21588,1
104,18767,0
105,19176,0
106,17006,0
107,17288,0
108,17192,0
109,17640,0


In [9]:
from sklearn.cluster import *
clustering_training = KMeans(n_clusters=5, random_state=0).fit(x1) #FeatureAgglomeration().fit(x1)#DBSCAN().fit(new_x1) #AgglomerativeClustering().fit(x1)#KMeans(n_clusters=5, random_state=0).fit(x1)

In [10]:
y_predicted = clustering_training.predict(t1)

In [12]:
len([x for x in y_predicted if x==0])

1500

In [65]:
[x for x in clustering_training.labels_ if x !=0]

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 2,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1